In [ ]:
!rm -rf ~/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-OCR-2*
!rm -rf ~/.cache/huggingface/modules/transformers_modules/deepseek-ai--DeepSeek-OCR-2*

!pip uninstall -y transformers tokenizers flash-attn
!pip uninstall -y torchvision torchaudio
!pip install --no-cache torch==2.6.0 --index-url https://download.pytorch.org/whl/cu118
!pip install --no-cache transformers==4.46.3 tokenizers==0.20.3
!pip install torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu118
!pip install einops addict easydict

In [ ]:
import torch
import torchvision
print(torch.__version__)       # Should be 2.6.0
print(torchvision.__version__) # Should be 0.21.0
import transformers
print(transformers.__version__) # Should be 4.46.3 from earlier fix

In [ ]:
!pip install num2words python-docx

In [ ]:
import os
import json
import logging
import torch
from num2words import num2words
from docx import Document
from docx.shared import Pt, Cm
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_ALIGN_PARAGRAPH
from openai import OpenAI
from transformers import AutoModel, AutoTokenizer
import time
import random
import gc
import re

# ================= НАСТРОЙКИ =================
API_KEY = "your-openrouter-api-key-here" 
BASE_URL = "https://openrouter.ai/api/v1"
model_name = 'deepseek-ai/DeepSeek-OCR-2'

os.environ["CUDA_VISIBLE_DEVICES"] = '0,1'
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

BATCH_FOLDERS = [
    "folder/path/one", 
    "folder/path/two",
    "folder/path/three"
]

OUTPUT_FILE = "GOTOVAYA_OPIS_FULL.docx"
TEMP_OUTPUT_DIR = "temp_ocr_results"
PROGRESS_FILE = "progress.json"  # Файл для сохранения прогресса

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# ================= ИНИЦИАЛИЗАЦИЯ МОДЕЛЕЙ =================
logging.info("Загрузка модели DeepSeek-OCR...")

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    use_safetensors=True
)
model = model.eval().cuda().to(torch.bfloat16)

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

In [ ]:
def clean_ocr_text(text):
    """Удаляем специальные токены разметки"""
    if not text:
        return ""
    
    text = re.sub(r'<\|ref\|>.*?<\|/ref\|>', '', text)
    text = re.sub(r'<\|det\|>\[\[.*?\]\]<\|/det\|>', '', text)
    text = re.sub(r'<\|grounding\|>', '', text)
    text = re.sub(r'<\|.*?\|>', '', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    return text.strip()


def collect_all_cases(batch_folders):
    """Собирает все папки с делами"""
    all_cases = []
    for batch_name in batch_folders:
        if not os.path.exists(batch_name):
            logging.warning(f"Папка '{batch_name}' не найдена! Пропускаю.")
            continue
        items = sorted(os.listdir(batch_name))
        for item in items:
            full_path = os.path.join(batch_name, item)
            if os.path.isdir(full_path) and not item.startswith('.'):
                all_cases.append(full_path)
    return all_cases


def run_ocr_on_folder(folder_path):
    """Читает ВСЕ фото внутри папки дела"""
    full_text = ""
    files = sorted(os.listdir(folder_path))
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    
    if not image_files:
        return None

    logging.info(f"--- Обработка: {folder_path} ({len(image_files)} фото) ---")
    
    if not os.path.exists(TEMP_OUTPUT_DIR):
        os.makedirs(TEMP_OUTPUT_DIR)

    result_file = os.path.join(TEMP_OUTPUT_DIR, "result.mmd")

    for filename in image_files:
        image_path = os.path.join(folder_path, filename)
        
        torch.cuda.empty_cache()
        gc.collect()
        
        if os.path.exists(result_file):
            os.remove(result_file)
        
        try:
            prompt = "<image>\n<|grounding|>Convert the document to markdown. "
            res = model.infer(tokenizer, prompt=prompt, image_file=image_path, 
                            output_path=TEMP_OUTPUT_DIR, base_size=1024, 
                            image_size=768, crop_mode=True, save_results=True)
            
            if os.path.exists(result_file):
                with open(result_file, 'r', encoding='utf-8') as f:
                    text_content = f.read()
                
                text_content = clean_ocr_text(text_content)
                
                if text_content:
                    full_text += f"\n\n=== ФАЙЛ: {filename} ===\n"
                    full_text += text_content
                    full_text += f"\n=== КОНЕЦ ФАЙЛА ===\n"
                    logging.info(f"✓ {filename}: получено {len(text_content)} символов")
                else:
                    logging.warning(f"✗ {filename}: пустой результат после очистки")
            else:
                logging.warning(f"✗ {filename}: result.mmd не создан")
            
        except Exception as e:
            logging.error(f"Ошибка OCR {filename}: {e}")
            torch.cuda.empty_cache()
            gc.collect()

    return full_text


def analyze_structure_with_api(ocr_text, max_retries=6):
    """Анализирует текст и возвращает структурированные данные"""
    
    ocr_text = clean_ocr_text(ocr_text)
    
    system_prompt = """
Ты — интеллектуальный архивариус. На входе — OCR текста папки/дела.

Твоя задача: составить запись для описи. Верни ТОЛЬКО JSON (без ```).

ПРАВИЛА:
1) Если есть признаки документов — НЕЛЬЗЯ писать, что документов нет.
2) Если это набор разнородных материалов — "Подборка документов (…виды документов…)".

ПОЛЯ JSON:
- "title": архивное название + в скобках виды документов
- "date": крайние даты (формат "2016" или "2016-2017"). Если не найдено — ""
- "pages": количество листов (число). Если не определено — ""
- "storage": срок хранения. По умолчанию "5 лет"
- "index": индекс/номер дела. Если не найден — ""

Формат ответа: ТОЛЬКО JSON-объект, без пояснений.
"""

    truncated_text = ocr_text[:120000]
    logging.info(f"Отправляем в API текст длиной {len(truncated_text)} символов")
    logging.info(f"Первые 500 символов:\n{truncated_text[:500]}")

    base_delay = 1.5

    for attempt in range(1, max_retries + 1):
        try:
            response = client.chat.completions.create(
                model="xiaomi/mimo-v2-flash",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Вот распознанный текст:\n\n{truncated_text}"},
                ],
                temperature=0.1,
                max_tokens=2500
            )

            content = response.choices[0].message.content
            logging.info(f"Ответ API: {content}")
            
            content = content.replace("```json", "").replace("```", "").strip()
            return json.loads(content)

        except Exception as e:
            logging.error(f"[API ERROR] Попытка {attempt}/{max_retries}: {e}")

            if attempt == max_retries:
                return {
                    "title": "Ошибка анализа",
                    "date": "",
                    "pages": "",
                    "storage": "5 лет",
                    "index": ""
                }

            delay = base_delay * (2 ** (attempt - 1))
            jitter = random.uniform(0.8, 1.2)
            time.sleep(delay * jitter)


# ================= СОХРАНЕНИЕ/ЗАГРУЗКА ПРОГРЕССА =================

def save_progress(table_rows, processed_paths):
    """Сохраняет прогресс в JSON файл"""
    data = {
        'table_rows': table_rows,
        'processed_paths': processed_paths
    }
    with open(PROGRESS_FILE, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    logging.info(f"💾 Прогресс сохранён: {len(table_rows)} записей")


def load_progress():
    """Загружает прогресс из JSON файла"""
    if os.path.exists(PROGRESS_FILE):
        try:
            with open(PROGRESS_FILE, 'r', encoding='utf-8') as f:
                data = json.load(f)
            logging.info(f"📂 Загружен прогресс: {len(data.get('table_rows', []))} записей")
            return data.get('table_rows', []), set(data.get('processed_paths', []))
        except Exception as e:
            logging.warning(f"Ошибка загрузки прогресса: {e}")
    return [], set()


# ================= СОЗДАНИЕ WORD ДОКУМЕНТА =================

def create_opis_document(table_rows, output_path):
    """Создаёт Word документ с описью БЕЗ шаблона"""
    
    doc = Document()
    
    # Настройка страницы (альбомная A4)
    section = doc.sections[0]
    section.page_width = Cm(29.7)
    section.page_height = Cm(21)
    section.left_margin = Cm(2)
    section.right_margin = Cm(1.5)
    section.top_margin = Cm(1.5)
    section.bottom_margin = Cm(1.5)
    
    # === ШАПКА ===
    def add_text(text, size=14, bold=False, center=False):
        para = doc.add_paragraph()
        if center:
            para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = para.add_run(text)
        run.font.size = Pt(size)
        run.font.name = 'Times New Roman'
        run.font.bold = bold
        return para
    
    add_text("Государственное автономное учреждение", center=True)
    add_text("«ПРИМЕР ПРИМЕР ПРИМЕР", center=True)
    
    doc.add_paragraph()
    
    # Блок УТВЕРЖДАЮ
    approve = doc.add_paragraph()
    approve.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    run_app = approve.add_run(
        "УТВЕРЖДАЮ\n"
        "РУКОВОДИТЕЛЬ службы\n"
        "____________И.И. ИВАНОВ\n"
        "«___»_____________ 2026 г."
    )
    run_app.font.size = Pt(12)
    run_app.font.name = 'Times New Roman'
    
    doc.add_paragraph()
    
    add_text("Опись № ____", size=16, bold=True, center=True)
    add_text("за 2010-2026 годы", size=14, center=True)
    
    doc.add_paragraph()
    
    # === ТАБЛИЦА ===
    table = doc.add_table(rows=1, cols=7)
    table.style = 'Table Grid'
    table.alignment = WD_TABLE_ALIGNMENT.CENTER
    
    # Заголовки
    headers = ['№ п/п', 'Индекс дела', 'Заголовок дела', 'Дата дела', 
               'Срок хранения', 'Кол-во листов', 'Прим.']
    
    header_row = table.rows[0]
    for i, header in enumerate(headers):
        cell = header_row.cells[i]
        para = cell.paragraphs[0]
        para.clear()
        run = para.add_run(header)
        run.font.bold = True
        run.font.size = Pt(10)
        run.font.name = 'Times New Roman'
        para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Нумерация колонок
    num_row = table.add_row()
    for i in range(7):
        cell = num_row.cells[i]
        para = cell.paragraphs[0]
        para.clear()
        run = para.add_run(str(i + 1))
        run.font.size = Pt(9)
        run.font.name = 'Times New Roman'
        para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # === ДАННЫЕ ===
    for row_data in table_rows:
        row = table.add_row()
        
        # Извлекаем значения
        values = [
            (str(row_data.get('num', '')), True),
            (str(row_data.get('index', '') or ''), True),
            (str(row_data.get('title', 'Не распознано')), False),
            (str(row_data.get('date', '') or ''), True),
            (str(row_data.get('storage', '5 лет') or '5 лет'), True),
            (str(row_data.get('pages', '') or ''), True),
            (str(row_data.get('note', '') or ''), False)
        ]
        
        for i, (value, center) in enumerate(values):
            cell = row.cells[i]
            para = cell.paragraphs[0]
            para.clear()
            run = para.add_run(value)
            run.font.size = Pt(10)
            run.font.name = 'Times New Roman'
            if center:
                para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    doc.add_paragraph()
    
    # === ИТОГОВАЯ ЗАПИСЬ ===
    total = len(table_rows)
    try:
        total_words = num2words(total, lang='ru')
    except:
        total_words = str(total)
    
    summary = doc.add_paragraph()
    run_sum = summary.add_run(
        f"В данный раздел описи внесено {total} ({total_words}) дел "
        f"с № 1 по № {total}, в том числе:\n"
        f"литерные номера: —\n"
        f"пропущенные номера: —"
    )
    run_sum.font.size = Pt(12)
    run_sum.font.name = 'Times New Roman'
    
    doc.add_paragraph()
    doc.add_paragraph()
    
    # Подписи
    signatures = doc.add_paragraph()
    run_sign = signatures.add_run(
        "Главный специалист _________________ И.О.Фамилия\n\n"
        "«_____»___________ 2026 г.\n\n\n"
        "Передал ______________________________________________ ед. хр.\n"
        "                    (цифрами и прописью)\n\n\n"
        "Главный специалист ООД _________________ И.О.Фамилия\n\n"
        "«_____»___________ 2026 г.\n\n\n"
        "Принял ______________________________________________ ед. хр.\n"
        "                    (цифрами и прописью)\n\n"
        "Наименование должности работника архива _________________ И.О.Фамилия\n\n"
        "«_____»___________ 2026 г."
    )
    run_sign.font.size = Pt(12)
    run_sign.font.name = 'Times New Roman'
    
    # Сохраняем
    doc.save(output_path)
    logging.info(f"✅ Word документ сохранён: {output_path}")


# ================= ГЛАВНАЯ ФУНКЦИЯ =================

def main():
    logging.info("Сканирование папок...")
    all_cases_paths = collect_all_cases(BATCH_FOLDERS)
    
    if not all_cases_paths:
        print("Не найдено ни одной папки с делами!")
        return

    print(f"Всего найдено дел для обработки: {len(all_cases_paths)}")
    
    # Загружаем прогресс если есть
    table_rows, processed_paths = load_progress()
    
    # Определяем стартовый номер
    start_num = len(table_rows) + 1
    
    if processed_paths:
        print(f"📂 Продолжаем с дела #{start_num} (уже обработано: {len(processed_paths)})")

    for idx, folder_path in enumerate(all_cases_paths, 1):
        
        # Пропускаем уже обработанные
        if folder_path in processed_paths:
            continue
        
        torch.cuda.empty_cache()
        gc.collect()
        
        markdown_text = run_ocr_on_folder(folder_path)
        
        if not markdown_text or len(markdown_text.strip()) < 50:
            logging.warning(f"Пустая папка или нет данных: {folder_path}")
            processed_paths.add(folder_path)
            continue
        
        # Сохраняем OCR для отладки
        debug_file = os.path.join(TEMP_OUTPUT_DIR, f"debug_{len(table_rows)+1}.txt")
        with open(debug_file, 'w', encoding='utf-8') as f:
            f.write(markdown_text)
            
        data = analyze_structure_with_api(markdown_text)
        
        row = {
            'num': len(table_rows) + 1,
            'index': data.get('index', ''),
            'title': data.get('title', 'Не распознано'),
            'date': data.get('date', ''),
            'storage': data.get('storage', '5 лет'),
            'pages': data.get('pages', ''),
            'note': ''
        }
        table_rows.append(row)
        processed_paths.add(folder_path)
        
        print(f"--> [{len(table_rows)}/{len(all_cases_paths)}] Готово: {row['title'][:80]}...")
        
        # ⭐ АВТОСОХРАНЕНИЕ после каждого дела!
        save_progress(table_rows, list(processed_paths))
        
        torch.cuda.empty_cache()
        gc.collect()

    # Финальное сохранение
    if table_rows:
        # Сохраняем JSON
        final_json = "final_records.json"
        with open(final_json, 'w', encoding='utf-8') as f:
            json.dump(table_rows, f, ensure_ascii=False, indent=2)
        print(f"💾 JSON сохранён: {final_json}")
        
        # Создаём Word
        try:
            create_opis_document(table_rows, OUTPUT_FILE)
            print(f"\n{'='*50}")
            print(f"✅ УСПЕХ! Опись создана: {OUTPUT_FILE}")
            print(f"📊 Обработано дел: {len(table_rows)}")
            print(f"{'='*50}")
        except Exception as e:
            logging.error(f"Ошибка создания Word: {e}")
            print(f"⚠️ Word не создан, но данные сохранены в {final_json}")
    else:
        print("❌ Нет данных для сохранения")


In [ ]:
main()

In [ ]:
# === ВОССТАНОВЛЕНИЕ ИЗ JSON (если нужно пересоздать Word) ===

def recreate_word_from_json(json_path="final_records.json", output_path="OPIS_RESTORED.docx"):
    """Пересоздаёт Word из сохранённого JSON"""
    with open(json_path, 'r', encoding='utf-8') as f:
        table_rows = json.load(f)
    
    print(f"📂 Загружено {len(table_rows)} записей из {json_path}")
    create_opis_document(table_rows, output_path)
    print(f"✅ Word создан: {output_path}")

# Раскомментируй если нужно:
# recreate_word_from_json("progress.json", "OPIS_FROM_PROGRESS.docx")